In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import folium
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import pickle
import io
import fiona

from PIL import Image
from tqdm.notebook import tqdm
from pyproj import Transformer
from shapely.geometry import LineString, Point

In [7]:
GRID = 'data/joined.geojson'
GRAPH = 'data/ntb_graph.pickle'
FASKES = 'data/rs_bpjs.xlsx'
RADIUS = 10000
RESULT = 'data/akses_rs_30.geojson'

In [3]:
with open(GRAPH, 'rb') as f:
    graph = pickle.load(f)

In [4]:
joined_gdf = gpd.read_file(GRID).to_crs('epsg:3857')

In [5]:
joined_gdf['centroid'] = joined_gdf['geometry'].centroid

In [10]:
faskes = pd.read_excel(FASKES, dtype=str)

In [11]:
faskes_ntb = faskes.loc[faskes['prov']=='NUSA TENGGARA BARAT']

In [12]:
amenities_node = gpd.GeoDataFrame(
    faskes_ntb,
    geometry=gpd.points_from_xy(
        faskes_ntb['lng'],
        faskes_ntb['lat']
    )
)
amenities_node = amenities_node.set_crs("EPSG:4326")

In [13]:
joined_gdf['centroid'] = joined_gdf['centroid'].to_crs('epsg:4326')

In [14]:
# Create a function to calculate the distance along the road network
def calculate_distance_along_network(graph, start_node, end_node, weight="length"):
    try:
        path = nx.shortest_path(graph, source=start_node, target=end_node, weight=weight)
        path_length = nx.shortest_path_length(graph, source=start_node, target=end_node, weight=weight)
        return path, path_length
    except nx.NetworkXNoPath:
        return [], float('inf')

In [15]:
trans_3857 = Transformer.from_crs('EPSG:4326', 'EPSG:3857')
trans_4326 = Transformer.from_crs('EPSG:3857', 'EPSG:4326')

In [16]:
facilities = []
distances = []

for i in tqdm(joined_gdf['centroid'], desc='GRID', position=0):
    # Find the closest hospital using Dijkstra's algorithm
    closest_facility = None
    closest_distance = float('inf')
    best_path = None

    poi_node = ox.distance.nearest_nodes(graph, i.x, i.y)
    buff = gpd.GeoDataFrame(
        geometry=[
            Point(
                trans_3857.transform(
                    i.y,
                    i.x
                )
            ).buffer(RADIUS)
        ]
    )
    buff.crs = 'EPSG:3857'
    buff = buff.to_crs('EPSG:4326')

    temp = gpd.sjoin(
        amenities_node,
        buff,
        how='inner',
        predicate='intersects'
    )

    if not temp.empty:
        for idx, facility in tqdm(
            temp.iterrows(),
            desc='LOOKING FOR FACILITIES',
            leave=False,
            total=temp.shape[0],
            position=1
        ):
            facility_coords = (facility['geometry'].y, facility['geometry'].x)

            # Find the nearest network node to the hospital
            facility_node = ox.distance.nearest_nodes(
                graph, facility_coords[1], facility_coords[0]
            )

            # Calculate the distance along the road network from the POI to the hospital
            path, distance = calculate_distance_along_network(
                graph, poi_node, facility_node, weight="length"
            )

            if distance < closest_distance:
                closest_distance = distance
                closest_facility = facility['kdppk']
                best_path = path

        facilities.append(closest_facility)
        distances.append(closest_distance)

    else:
        facilities.append(closest_facility)
        distances.append(closest_distance)

GRID:   0%|          | 0/6146 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
# Add the list as a new column
joined_gdf['facility'] = facilities
joined_gdf['distance'] = distances
joined_gdf.loc[joined_gdf['distance']==np.inf, 'distance'] = 100000

In [7]:
pd.merge(
    joined_gdf,
    faskes_ntb[['kdppk', 'nmppk', 'total_doctors']],
    left_on='facility',
    right_on='kdppk',
    how='left'
).to_file(RESULT)